# **Import Statements**

In [0]:
import os
import cv2
import sys

# Shared Class files required for program execution 
#sys.path.append('/content/drive/My Drive/CV_task2/shared_files') # if using Google Colab else comment
from centroiddetails import CentroidDetails
from trackedids import TrackedIDs
from yolodetection import YoloDetection
#from google.colab.patches import cv2_imshow

# **Global Variables**

In [0]:
# Given Image file directory
img_sequence_dir = '/content/drive/My Drive/CV_GroupProject/sequence_imgs/'

# Task1 
# Output Image files directory
task1_output_img_dir = '/content/drive/My Drive/Outputs/project/task_1_out_images/'
# Output Video file directory
task1_video_dir = '/content/drive/My Drive/Outputs/project/task_1_video.avi'

#Task2 
# Output Image files directory
task2_output_img_dir = '/content/drive/My Drive/Outputs/project/task_2_out_images/'
# Output Video file directory
task2_video_dir = '/content/drive/My Drive/Outputs/project/task_2_video.avi'

#Task3
# Output Image files directory
task3_output_img_dir = '/content/drive/My Drive/Outputs/project/task_3_out_images/'
# Output Video file directory
task3_video_dir = '/content/drive/My Drive/Outputs/project/task_3_video.avi'

# **Utility Functions**


# Create Video from the Output Image Files

In [0]:
# Create video from the image files
def create_video(input_img_dir, output_video_dir):
  img_array = []
  img_id = '000000' 
  for count in range(1,794):
      input_img = (input_img_dir+img_id[len(str(count)):]+str(count)+'.jpg')
      img = cv2.imread(input_img)
      if img is None :
        continue
      height, width, _ = img.shape
      size = (width,height)
      img_array.append(img)
  out = cv2.VideoWriter(output_video_dir,cv2.VideoWriter_fourcc(*'DIVX'), 3, size)

  for i in range(len(img_array)):
      out.write(img_array[i])
  out.release()

# Task 1

# a) detect all the moving pedestrians and draw a bounding box around each of them
# b) draw the trajectory (track / path) of each moving pedestrian
# c) print (as an output to the terminal, or directly on the image sequence / video) the real- time count of all the moving pedestrians detected since start of the video frame 1.

In [0]:
def task_1():
  object_count = 0
  obj_centroid = CentroidDetails(max_dist_apart=150, max_obj_retention=75)
  yolo = YoloDetection(confThreshold=0.65,nmsThreshold=.40,inpWidth=576,inpHeight=768)
  trackedIDs = {}
  colour_code = {}
  colour_list = [(0,0,0),(255,255,255),(255,0,0),(0,255,0),(0,0,255),(255,255,0),(0,255,255),(255,0,255),(128,0,0),(128,128,0),(0,128,0),(128,0,128),(0,128,128),(0,0,128),(255,127,80),(205,92,92),(184,134,11),(143,188,143),(0,255,255),(100,149,237),(135,206,235),(199,21,133),(245,245,220),(139,69,19),(244,164,96),(255,228,225),(253,245,230),(245,255,250),(240,255,240),(240,255,255),(192,192,192),(192,192,192),(128,128,128)]

  img_id = '000000' 
  for count in range(1,len(os.listdir(img_sequence_dir))):
    img = (img_sequence_dir+img_id[len(str(count)):]+str(count)+'.jpg')
    outfile = (task1_output_img_dir +img_id[len(str(count)):]+str(count)+'.jpg')
    frame = cv2.imread(img)
    if frame is None:
      continue
    else:
      object_box_dim = yolo.boundingBoxList(frame)
    objects,pedestrian_count = obj_centroid.update(object_box_dim)
    if objects == 0:  # Skip the frame
      continue

    for (ID, centroid) in objects.items():
      to = trackedIDs.get(ID, None)
      if to is None:
        object_count = object_count +1
        if object_count >=len(colour_list):
          object_count = 0
        to = TrackedIDs(ID, centroid)
        colour_code[ID] = colour_list[object_count]
      else:
        to.centroids.append(centroid)
        line_size = len(to.centroids)
        for m in range(line_size-1):
          start_point = to.centroids[m][0],to.centroids[m][1]
          end_point = to.centroids[m+1][0],to.centroids[m+1][1]
          cv2.line(frame, start_point, end_point, colour_code[ID], 1)
      trackedIDs[ID] = to

      text = "{}".format(ID)
      cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
      cv2.circle(frame, (centroid[0], centroid[1]), 4, colour_code[ID], -1)
    
    text_val= "Total Num of Pedestrians"
    ped_count = pedestrian_count
    text = "{}: {}".format(text_val, ped_count)
    height, _ = frame.shape[:2]
    cv2.putText(frame, text, (10, height - (40)),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
    #cv2_imshow(frame)
    cv2.imwrite(outfile, frame)
  cv2.destroyAllWindows()


# Task 2:
# You should allow the user to draw a rectangular bounding box within the whole window (hint: use OpenCV to draw) on the first frame / image. Then print (as output to the terminal, or print on the image sequence / video) the real-time counts of:
# a) pedestrians who enter the bounding box
# b) pedestrians who move out of the bounding box

In [0]:
def task_2(X,Y,Width,Height):
  obj_centroid = CentroidDetails(max_dist_apart=150, max_obj_retention=75)
  yolo = YoloDetection(confThreshold=0.65,nmsThreshold=.40,inpWidth=576,inpHeight=768)
  trackedIDs = {}
  totalDown = 0
  totalUp = 0
  enteredObject =[]
  exitedObject = []
  colour_code = {}

  img_id = '000000' 
  for count in range(1,len(os.listdir(img_sequence_dir))):
    img = (img_sequence_dir+img_id[len(str(count)):]+str(count)+'.jpg')
    outfile = (task2_output_img_dir+ img_id[len(str(count)):]+str(count)+'.jpg')
    frame = cv2.imread(img)
    if frame is None:
      continue
    else:
      object_box_dim = yolo.boundingBoxList(frame)
    cv2.rectangle(frame, (X, Y), (Width, Height), (255, 0, 0), 1)
    objects,pedestrian_count = obj_centroid.update(object_box_dim)
    if objects == 0:
      continue
    for (ID, centroid) in objects.items():
      to = trackedIDs.get(ID, None)
      if to is None:
        to = TrackedIDs(ID, centroid)
      else:
        to.centroids.append(centroid)
        if ID in enteredObject and ID in exitedObject :
          cv2.circle(frame, (centroid[0], centroid[1]), 4, colour_code[ID], -1)
          continue
        else:
          if not to.flagged:
            if centroid[0] > X and centroid[0] <Width and centroid[1] > Y and centroid[1] < Height :
              enteredObject.append(ID)
              totalUp += 1
              to.flagged = True
              cv2.circle(frame, (centroid[0], centroid[1]), 4, (0,255,0), -1)
              colour_code[ID] = (0,255,0)
          elif to.flagged and centroid[0] > X and centroid[0] <Width and centroid[1] > Y and centroid[1] < Height :
            cv2.circle(frame, (centroid[0], centroid[1]), 4, colour_code[ID], -1)
          elif to.flagged and (not(centroid[0] > X and centroid[0] <Width and centroid[1] > Y and centroid[1] < Height) and not(ID in exitedObject)) :
            exitedObject.append(ID)
            totalDown += 1
            colour_code[ID] =  (0,0,255)
            cv2.circle(frame, (centroid[0], centroid[1]), 4, colour_code[ID], -1)
      trackedIDs[ID] = to
      text = "{}".format(ID)
    info = [("Entered", totalUp),("Exited", totalDown)]

    for (i, (name, value)) in enumerate(info):
      text = "{}: {}".format(name, value)
      height, _ = frame.shape[:2]
      cv2.putText(frame, text, (10, height - ((i * 20) + 20)),cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
    #cv2_imshow(frame)
    cv2.imwrite(outfile, frame)
  cv2.destroyAllWindows()

# **Main Execution**

In [0]:
task_1()
create_video(task1_output_img_dir, task1_video_dir)
task_2( X = 580,Y = 80,Width = 730,Height = 400) # Please provide the bounding box dimensions here 
create_video(task2_output_img_dir, task2_video_dir)